In [2]:
import os

In [3]:
%pwd

'e:\\Sem-7-Project\\My-Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\Sem-7-Project\\My-Text-Summarizer'

In [6]:
# Creating Entity (Step 3 : From readme.md)

from dataclasses import dataclass
from pathlib import Path
# Values from yamll....
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
# Faced an issue with project Environment.. By mistake I have reInitialized the python environment so I have to Reinstall all requirements to create project (as Package as below imported). 

from textSummarizer.constants import * # To access constants...
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
# Step 4 : (From readme.md) Configuration manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  # Used Constants..
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])  # Use common.py file's functions..

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])   # Using Config.yaml file's variables..

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config  # Returning custom data Using DataIngestionConfig

In [9]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [10]:
# Step : 5 (readme.md) Update the components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # To download the data file from URL
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists with of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Step 6 : Update the pipeline (readme.md)..

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-22 09:18:41,545: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-22 09:18:41,555: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-22 09:18:41,557: INFO: common: created directory at: artifacts]
[2023-07-22 09:18:41,558: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-22 09:18:52,173: INFO: 3849362711: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 62BE:5D26:1A8FEF:1F8524:64BB5198
Accept-Ranges: bytes
Date: Sat, 22 Jul 2023 03:48:43 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4725-BOM
X-Cache: MISS